In [1]:
!nvidia-smi

Mon Sep  7 14:28:43 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import cv2
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers

import string

path = '/content/gdrive/My Drive/Dacon/ComputerVision/'

path_train = path + 'train.csv'
path_test = path + 'test.csv'
path_submission = path + 'submission.csv'

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


InceptionResnetV2_bicubic224
-----------

In [5]:
seed = 0
tf.random.set_seed(seed)
np.random.seed(seed)

ImageGen_coeff = 10
epochs_num = 30
verbose = 2
N_dim = 224
ModelName = 'InceptionResnetV2_bicubic224'

In [ ]:
def normalization(df):
    for row in range(len(df)):
        maxi = df.iloc[row].max()
        mini = df.iloc[row].min()
        if maxi == mini:
            df.iloc[row] = df.iloc[row].apply(lambda x : 0)
        else:
            df.iloc[row] = df.iloc[row].apply(lambda x : (x - mini)/(maxi - mini))
    x = df.values
    if not np.any(x > 1.0) and not np.any(x < 0) and not np.any(np.isnan(x)):
        print('Boundary Clear')      
    return df

def ComputerVision_Dataset():
    train_data = pd.read_csv('/content/gdrive/My Drive/Dacon/ComputerVision/train.csv')
    train_data_pixel = pd.read_csv('/content/gdrive/My Drive/Dacon/ComputerVision/train_bicubic'+str(N_dim)+'.csv')

    letter_hash = dict(zip(string.ascii_uppercase, [[1 if i == j else 0 for j in range(26)] for i in range(26)]))

    pix = train_data_pixel.iloc[:, 1:]
    pix = normalization(pix).values.reshape(-1, N_dim, N_dim, 1)

    fix = train_data.iloc[:, 1:3].values

    datagenerator = ImageDataGenerator(rotation_range=10, zoom_range=0.10, width_shift_range=0.1,
                                       height_shift_range=0.1)
    gen = datagenerator.flow(pix, fix, shuffle=False, batch_size=32)
    pixel, fixed, batch_index, limit = [], [], 0, 64 * ImageGen_coeff
    while batch_index <= limit:
        try:
            data = gen.next()
            pixel += list(data[0])
            fixed += list(data[1])
            batch_index += 1
        except:
            print("ImageGeneratorError")
            break
    fixed_train = np.asarray(fixed)
    X_train_pixel = np.asarray(pixel)
    Y_train = to_categorical(fixed_train[:, 0], 10)
    X_train_label = np.asarray([letter_hash[letter] for letter in fixed_train[:, 1]])


    return X_train_label, X_train_pixel, Y_train


X_train_label, X_train_pixel, Y_train = ComputerVision_Dataset()

Boundary Clear


In [ ]:
def train_test_model(X_train_label, X_train_pixel, Y_train):
    print(ModelName)
    with tf.device('/device:GPU:0'):
        resnet50 = tf.keras.applications.InceptionResNetV2(include_top=False, weights=None, input_tensor=None, input_shape=(N_dim,N_dim,1),pooling=None)
        cnn_mid = layers.GlobalAveragePooling2D()(resnet50.output)
        cnn_out = layers.Dense(128, activation = 'relu')(cnn_mid)

        dense_input = layers.Input(shape=(26,))
        dense_mid = layers.Dense(52, activation = 'relu')(dense_input)
        dense_output = layers.Dense(52, activation = 'relu')(dense_mid)
        dense_model = tf.keras.Model(inputs=dense_input, outputs=dense_output)

        concatenated = layers.concatenate([cnn_out, dense_model.output])
        concatenated = layers.Dense(32, activation='relu')(concatenated)
        concat_output = layers.Dense(10, activation='softmax')(concatenated)
        concat_model = tf.keras.models.Model([resnet50.input, dense_input], concat_output)
        concat_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
        history = concat_model.fit([X_train_pixel, X_train_label], Y_train, epochs=epochs_num,
                            verbose=verbose)
    concat_model.save(path + ModelName + '.h5')
    print(
        f"CNN: Epochs={epochs_num:d}, " +
        f"Train accuracy={max(history.history['accuracy']):.5f}, "
    )
    return concat_model

model = train_test_model(X_train_label, X_train_pixel, Y_train)

InceptionResnetV2_bicubic224
Epoch 1/30
641/641 - 211s - loss: 0.7779 - accuracy: 0.7383
Epoch 2/30
641/641 - 211s - loss: 0.2600 - accuracy: 0.9129
Epoch 3/30
641/641 - 211s - loss: 0.1604 - accuracy: 0.9453
Epoch 4/30
641/641 - 211s - loss: 0.1047 - accuracy: 0.9653
Epoch 5/30
641/641 - 211s - loss: 0.0848 - accuracy: 0.9717
Epoch 6/30
641/641 - 211s - loss: 0.0691 - accuracy: 0.9769
Epoch 7/30
641/641 - 211s - loss: 0.0643 - accuracy: 0.9798
Epoch 8/30
641/641 - 211s - loss: 0.0449 - accuracy: 0.9852
Epoch 9/30
641/641 - 211s - loss: 0.0489 - accuracy: 0.9854
Epoch 10/30
641/641 - 211s - loss: 0.0437 - accuracy: 0.9862
Epoch 11/30
641/641 - 211s - loss: 0.0327 - accuracy: 0.9908
Epoch 12/30
641/641 - 211s - loss: 0.0327 - accuracy: 0.9905
Epoch 13/30
641/641 - 212s - loss: 0.0375 - accuracy: 0.9881
Epoch 14/30
641/641 - 212s - loss: 0.0249 - accuracy: 0.9923
Epoch 15/30
641/641 - 212s - loss: 0.0317 - accuracy: 0.9899
Epoch 16/30
641/641 - 212s - loss: 0.0315 - accuracy: 0.9908
Epoc

pipeline

In [ ]:
def normalization(pixel_data):
    pixel_data = pixel_data.reshape(224 * 224)
    maxi = max(pixel_data)
    mini = min(pixel_data)
    if maxi == mini:
        pixel_data = np.array(list(map(lambda x: 0, pixel_data)))
    else:
        pixel_data = np.array(list(map(lambda x: (x - mini) / (maxi - mini), pixel_data)))
    return pixel_data.reshape(224,224,1)

class ComputerVision_Data_test():
    def __init__(self, dataframe, batch_size):
        self.pixel = dataframe.iloc[:, 2:].values.astype('int32').reshape(-1, 28, 28, 1)
        letter_hash = dict(zip(string.ascii_uppercase, [[1 if i == j else 0 for j in range(26)] for i in range(26)]))
        self.label =  np.array([letter_hash[letter] for letter in dataframe.iloc[:, 1]])
        self.batch_size = batch_size
        if len(dataframe) % batch_size == 0:
            self.length = len(dataframe) // batch_size
        else:
            self.length = len(dataframe) // batch_size + 1

    def __getitem__(self, idx):
        i = idx * self.batch_size
        if i + self.batch_size > len(self.pixel):
            index_window = [j for j in range(i, len(self.label))]
            target = self.pixel[i:]
        else:
            index_window = [j for j in range(i, i+self.batch_size)]
            target = self.pixel[i:i+self.batch_size]
        with tf.device('/device:GPU:0'):
            transformed = tf.image.resize(target, [224,224], method=tf.image.ResizeMethod.BICUBIC).numpy()
        pixel = np.asarray([normalization(elt) for elt in transformed])
        label = np.asarray([self.label[k] for k in index_window])

        return [pixel, label]

    def __len__(self):
        return self.length


test_data = pd.read_csv('/content/gdrive/My Drive/Dacon/ComputerVision/test.csv')
test_dataset = ComputerVision_Data_test(test_data, 1000)

In [ ]:
print(len(test_data),len(test_dataset))

20480 21


In [ ]:
model = tf.keras.models.load_model(path + ModelName + '.h5')

In [ ]:
def get_progressbar_str(progress):
    MAX_LEN = 30
    BAR_LEN = int(MAX_LEN * progress)
    return ('Progress:[' + '=' * BAR_LEN +
            ('>' if BAR_LEN < MAX_LEN else '') +
            ' ' * (MAX_LEN - BAR_LEN) +
            '] %.1f%%' % (progress * 100.))

def split_testing(model,ModelName, test_dataset, batch_idxs):
    batch_size = test_dataset.batch_size
    submission = pd.read_csv(path_submission)
    inc = 0
    L = len(batch_idxs)
    for batch_idx in batch_idxs:
        target = test_dataset[batch_idx]
        real_idxs = [i for i in range(batch_size * batch_idx, batch_size * batch_idx + len(target[0])) ]
        with tf.device('/device:GPU:0'):
            predicted = model.predict(target)
        res = [int(np.argmax(x)) for x in predicted]
        for idx_temp, real in enumerate(real_idxs):
            submission.iloc[real,1] = res[idx_temp]
        inc += 1
        sys.stderr.write('\r\033[K' + get_progressbar_str(inc/L))
        sys.stderr.flush()
    sys.stderr.write('\n')
    sys.stderr.flush()
    
    submission.to_csv(path+'submission_'+ModelName+'.csv',index = False)

    return submission



In [ ]:
#1
batch_idxs = [i for i in range(0, len(test_dataset))]
submission = split_testing(model,ModelName, test_dataset, batch_idxs)

Progress:[==============================] 100.0%


EfficientNetB7
-----------

In [ ]:
seed = 0
tf.random.set_seed(seed)
np.random.seed(seed)

ImageGen_coeff = 10
epochs_num = 30
verbose = 1
N_dim = 224
ModelName = 'EfficientNetB5_bicubic224'

In [ ]:
def train_test_model(X_train_label, X_train_pixel, Y_train):
    print(ModelName)
    with tf.device('/device:GPU:0'):
        resnet50 = tf.keras.applications.EfficientNetB5(include_top=False, weights=None, input_tensor=None, input_shape=(N_dim,N_dim,1),pooling=None)
        cnn_mid = layers.GlobalAveragePooling2D()(resnet50.output)
        cnn_out = layers.Dense(128, activation = 'relu')(cnn_mid)

        dense_input = layers.Input(shape=(26,))
        dense_mid = layers.Dense(52, activation = 'relu')(dense_input)
        dense_output = layers.Dense(52, activation = 'relu')(dense_mid)
        dense_model = tf.keras.Model(inputs=dense_input, outputs=dense_output)

        concatenated = layers.concatenate([cnn_out, dense_model.output])
        concatenated = layers.Dense(32, activation='relu')(concatenated)
        concat_output = layers.Dense(10, activation='softmax')(concatenated)
        concat_model = tf.keras.models.Model([resnet50.input, dense_input], concat_output)
        concat_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
        #print(concat_model.summary())
        history = concat_model.fit([X_train_pixel, X_train_label], Y_train, epochs=epochs_num,
                            verbose=verbose)
    concat_model.save(path + ModelName + '.h5')
    print(
        f"CNN: Epochs={epochs_num:d}, " +
        f"Train accuracy={max(history.history['accuracy']):.5f}, "
    )
    return concat_model

model = train_test_model(X_train_label, X_train_pixel, Y_train)

EfficientNetB5_bicubic224
Epoch 1/30
641/641 [==============================] - 278s 434ms/step - loss: 1.5625 - accuracy: 0.4528
Epoch 2/30
641/641 [==============================] - 277s 433ms/step - loss: 0.5825 - accuracy: 0.8140
Epoch 3/30
641/641 [==============================] - 276s 431ms/step - loss: 0.3088 - accuracy: 0.8994
Epoch 4/30
641/641 [==============================] - 277s 432ms/step - loss: 0.2003 - accuracy: 0.9351
Epoch 5/30
641/641 [==============================] - 278s 433ms/step - loss: 0.1353 - accuracy: 0.9575
Epoch 6/30
641/641 [==============================] - 277s 433ms/step - loss: 0.1389 - accuracy: 0.9554
Epoch 7/30
641/641 [==============================] - 278s 433ms/step - loss: 0.1037 - accuracy: 0.9674
Epoch 8/30
641/641 [==============================] - 277s 433ms/step - loss: 0.0839 - accuracy: 0.9759
Epoch 9/30
641/641 [==============================] - 277s 433ms/step - loss: 0.0693 - accuracy: 0.9787
Epoch 10/30
641/641 [=================

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
def normalization(pixel_data):
    pixel_data = pixel_data.reshape(224 * 224)
    maxi = max(pixel_data)
    mini = min(pixel_data)
    if maxi == mini:
        pixel_data = np.array(list(map(lambda x: 0, pixel_data)))
    else:
        pixel_data = np.array(list(map(lambda x: (x - mini) / (maxi - mini), pixel_data)))
    return pixel_data.reshape(224,224,1)

class ComputerVision_Data_test():
    def __init__(self, dataframe, batch_size):
        self.pixel = dataframe.iloc[:, 2:].values.astype('int32').reshape(-1, 28, 28, 1)
        letter_hash = dict(zip(string.ascii_uppercase, [[1 if i == j else 0 for j in range(26)] for i in range(26)]))
        self.label =  np.array([letter_hash[letter] for letter in dataframe.iloc[:, 1]])
        self.batch_size = batch_size
        if len(dataframe) % batch_size == 0:
            self.length = len(dataframe) // batch_size
        else:
            self.length = len(dataframe) // batch_size + 1

    def __getitem__(self, idx):
        i = idx * self.batch_size
        if i + self.batch_size > len(self.pixel):
            index_window = [j for j in range(i, len(self.label))]
            target = self.pixel[i:]
        else:
            index_window = [j for j in range(i, i+self.batch_size)]
            target = self.pixel[i:i+self.batch_size]
        with tf.device('/device:GPU:0'):
            transformed = tf.image.resize(target, [224,224], method=tf.image.ResizeMethod.BICUBIC).numpy()
        pixel = np.asarray([normalization(elt) for elt in transformed])
        label = np.asarray([self.label[k] for k in index_window])

        return [pixel, label]

    def __len__(self):
        return self.length


test_data = pd.read_csv('/content/gdrive/My Drive/Dacon/ComputerVision/test.csv')
test_dataset = ComputerVision_Data_test(test_data, 1000)

In [ ]:
def get_progressbar_str(progress):
    MAX_LEN = 30
    BAR_LEN = int(MAX_LEN * progress)
    return ('Progress:[' + '=' * BAR_LEN +
            ('>' if BAR_LEN < MAX_LEN else '') +
            ' ' * (MAX_LEN - BAR_LEN) +
            '] %.1f%%' % (progress * 100.))

def split_testing(model,ModelName, test_dataset, batch_idxs):
    batch_size = test_dataset.batch_size
    submission = pd.read_csv(path_submission)
    inc = 0
    L = len(batch_idxs)
    for batch_idx in batch_idxs:
        target = test_dataset[batch_idx]
        real_idxs = [i for i in range(batch_size * batch_idx, batch_size * batch_idx + len(target[0])) ]
        with tf.device('/device:GPU:0'):
            predicted = model.predict(target)
        res = [int(np.argmax(x)) for x in predicted]
        for idx_temp, real in enumerate(real_idxs):
            submission.iloc[real,1] = res[idx_temp]
        inc += 1
        sys.stderr.write('\r\033[K' + get_progressbar_str(inc/L))
        sys.stderr.flush()
    sys.stderr.write('\n')
    sys.stderr.flush()
    
    submission.to_csv(path+'submission_'+ModelName+'.csv',index = False)

    return submission

#2
batch_idxs = [i for i in range(0, len(test_dataset))]
submission = split_testing(model,ModelName, test_dataset, batch_idxs)

Progress:[==============================] 100.0%


InceptionResnetV2_bicubic224_VAE
---------------

In [ ]:
seed = 0
tf.random.set_seed(seed)
np.random.seed(seed)

ImageGen_coeff = 10
epochs_num = 30
verbose = 2
N_dim = 224
ModelName = 'InceptionResnetV2_bicubic224_VAE'

In [ ]:
def normalization(df):
    for row in range(len(df)):
        maxi = df.iloc[row].max()
        mini = df.iloc[row].min()
        if maxi == mini:
            df.iloc[row] = df.iloc[row].apply(lambda x : 0)
        else:
            df.iloc[row] = df.iloc[row].apply(lambda x : (x - mini)/(maxi - mini))
    x = df.values
    if not np.any(x > 1.0) and not np.any(x < 0) and not np.any(np.isnan(x)):
        print('Boundary Clear')      
    return df

def ComputerVision_Dataset():
    train_data = pd.read_csv('/content/gdrive/My Drive/Dacon/ComputerVision/train.csv')
    train_data_pixel = pd.read_csv('/content/gdrive/My Drive/Dacon/ComputerVision/train_bicubic'+str(N_dim)+'.csv')

    letter_hash = dict(zip(string.ascii_uppercase, [[1 if i == j else 0 for j in range(26)] for i in range(26)]))

    pix = train_data_pixel.iloc[:, 1:]
    pix = normalization(pix).values.reshape(-1, N_dim, N_dim, 1)

    fix = train_data.iloc[:, 1:3].values

    datagenerator = ImageDataGenerator(rotation_range=10, zoom_range=0.10, width_shift_range=0.1,
                                       height_shift_range=0.1)
    gen = datagenerator.flow(pix, fix, shuffle=False, batch_size=32)
    pixel, fixed, batch_index, limit = [], [], 0, 64 * ImageGen_coeff
    while batch_index <= limit:
        try:
            data = gen.next()
            pixel += list(data[0])
            fixed += list(data[1])
            batch_index += 1
        except:
            print("ImageGeneratorError")
            break
    fixed_train = np.asarray(fixed)
    X_train_pixel = np.asarray(pixel)
    Y_train = to_categorical(fixed_train[:, 0], 10)
    X_train_label = np.asarray([letter_hash[letter] for letter in fixed_train[:, 1]])


    return X_train_label, X_train_pixel, Y_train, [fix[:,1], pix, fix[:,0]]


X_train_label, X_train_pixel, Y_train, data_gen_for_vae = ComputerVision_Dataset()

Boundary Clear


In [ ]:
import tensorflow_probability as tfp
tfd = tfp.distributions
tfpl = tfp.layers

class VAE_MNIST(keras.Model):
    
    def __init__(self, dim_z, kl_weight=1, name="autoencoder", **kwargs):
        super(VAE_MNIST, self).__init__(name=name, **kwargs)
        self.dim_x = (224, 224, 1)
        self.dim_z = dim_z
        self.encoder = self.encoder_z()
        self.decoder = self.decoder_x()
        self.kl_weight = kl_weight
        
    # Sequential API encoder
    def encoder_z(self):
        encoder = [layers.InputLayer(input_shape=self.dim_x)]
        encoder.append(layers.Conv2D(filters=32, kernel_size=3, strides=(2,2), 
                                  padding='valid', activation='relu'))
        encoder.append(layers.Conv2D(filters=32, kernel_size=3, strides=(2,2), 
                                  padding='valid', activation='relu'))
        encoder.append(layers.Conv2D(filters=64, kernel_size=3, strides=(2,2), 
                                  padding='valid', activation='relu'))
        encoder.append(layers.Conv2D(filters=64, kernel_size=3, strides=(2,2), 
                                  padding='valid', activation='relu'))
        encoder.append(layers.Conv2D(filters=128, kernel_size=3, strides=(2,2), 
                                  padding='valid', activation='relu'))
        encoder.append(layers.GlobalAveragePooling2D())
        encoder.append(layers.Flatten())
        # *2 because number of parameters for both mean and (raw) standard deviation
        encoder.append(layers.Dense(self.dim_z*2, activation=None))
        return keras.Sequential(encoder)
    
    def encode(self, x_input):
        mu, rho = tf.split(self.encoder(x_input), num_or_size_splits=2, axis=1)
        sd = tf.math.log(1+tf.math.exp(rho))
        z_sample = mu + sd * tf.random.normal(shape=(self.dim_z,))
        return z_sample, mu, sd
    
    # Sequential API decoder
    def decoder_x(self):
        decoder = [layers.InputLayer(input_shape=self.dim_z)]
        decoder.append(layers.Dense(7*7*128, activation=None))
        decoder.append(layers.Reshape((7,7,128)))
        decoder.append(layers.Conv2DTranspose(filters=128, kernel_size=3, strides=2, 
                                           padding='same', activation='relu'))
        decoder.append(layers.Conv2DTranspose(filters=64, kernel_size=3, strides=2, 
                                           padding='same', activation='relu'))
        decoder.append(layers.Conv2DTranspose(filters=64, kernel_size=3, strides=2, 
                                           padding='same', activation='relu'))
        decoder.append(layers.Conv2DTranspose(filters=32, kernel_size=3, strides=2, 
                                           padding='same', activation='relu'))
        decoder.append(layers.Conv2DTranspose(filters=32, kernel_size=3, strides=2, 
                                           padding='same', activation='relu'))
        decoder.append(layers.Conv2DTranspose(filters=1, kernel_size=3, strides=1, 
                                           padding='same'))
        return keras.Sequential(decoder, name='decoder')
    
    def call(self, x_input):
        z_sample, mu, sd = self.encode(x_input)
        kl_divergence = tf.math.reduce_mean(- 0.5 * 
                tf.math.reduce_sum(1+tf.math.log(
                tf.math.square(sd))-tf.math.square(mu)-tf.math.square(sd), axis=1))
        x_logits = self.decoder(z_sample)
        # VAE_MNIST is inherited from keras.Model, thus have class method add_loss()
        self.add_loss(self.kl_weight * kl_divergence)
        return x_logits
    
# custom loss function with tf.nn.sigmoid_cross_entropy_with_logits
def custom_sigmoid_cross_entropy_loss_with_logits(x_true, x_recons_logits):
    raw_cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(
                                            labels=x_true, logits=x_recons_logits)
    neg_log_likelihood = tf.math.reduce_sum(raw_cross_entropy, axis=[1, 2, 3])
    return tf.math.reduce_mean(neg_log_likelihood)

  
####################   The following code shows how to train the model   ####################
# set hyperparameters
epochs = 30
batch_size = 32
lr = 0.0001
latent_dim=16
kl_w=3
# compile and train keras.Model
with tf.device('/device:GPU:0'):
    vae = VAE_MNIST(dim_z=latent_dim, kl_weight=kl_w)
    vae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), 
                loss=custom_sigmoid_cross_entropy_loss_with_logits)
    train_history = vae.fit(x=X_train_pixel, y=X_train_pixel, batch_size=batch_size, epochs=epochs)


Epoch 1/30
641/641 [==============================] - 17s 27ms/step - loss: 24433.8730
Epoch 2/30
641/641 [==============================] - 17s 27ms/step - loss: 22689.7246
Epoch 3/30
641/641 [==============================] - 17s 27ms/step - loss: 22601.5312
Epoch 4/30
641/641 [==============================] - 17s 27ms/step - loss: 22437.3008
Epoch 5/30
641/641 [==============================] - 17s 26ms/step - loss: 22071.2754
Epoch 6/30
641/641 [==============================] - 17s 27ms/step - loss: 21793.6934
Epoch 7/30
641/641 [==============================] - 17s 27ms/step - loss: 21645.2246
Epoch 8/30
641/641 [==============================] - 17s 26ms/step - loss: 21536.3613
Epoch 9/30
641/641 [==============================] - 17s 27ms/step - loss: 21432.8535
Epoch 10/30
641/641 [==============================] - 17s 27ms/step - loss: 21338.4258
Epoch 11/30
641/641 [==============================] - 17s 26ms/step - loss: 21256.9727
Epoch 12/30
641/641 [====================

In [ ]:
raw_data_label, raw_data_pixel, raw_data_ans = data_gen_for_vae
raw_data_ans = to_categorical(raw_data_ans, 10)
letter_hash = dict(zip(string.ascii_uppercase, [[1 if i == j else 0 for j in range(26)] for i in range(26)]))
raw_data_label = np.asarray([letter_hash[letter] for letter in raw_data_label])
with tf.device('/device:GPU:0'):
    generated_data = vae.predict(raw_data_pixel)
print(raw_data_label.shape, raw_data_pixel.shape, raw_data_ans.shape)
print(generated_data.shape, X_train_pixel.shape)
print(X_train_label.shape, Y_train.shape)

(2048, 26) (2048, 224, 224, 1) (2048, 10)
(2048, 224, 224, 1) (22560, 224, 224, 1)
(20512, 26) (20512, 10)


In [ ]:
def normalization_2(nump):
    nump = nump.reshape((N_dim*N_dim))
    maxi = max(nump)
    mini = min(nump)
    div = maxi - mini
    if maxi == mini:
        nump = np.zeros(nump.shape)
    else:
        nump = (nump - mini)/div
  
    return nump.reshape(N_dim,N_dim,1)

for i in range(generated_data.shape[0]):
    generated_data[i] = normalization_2(generated_data[i])

print('VAE generated data normalization complete.')

VAE generated data normalization complete.


In [ ]:
#X_train_pixel = np.concatenate((X_train_pixel, generated_data), axis = 0)
X_train_label = np.concatenate((X_train_label, raw_data_label), axis = 0)
Y_train = np.concatenate((Y_train, raw_data_ans), axis = 0)

print(generated_data.shape, X_train_pixel.shape)
print(X_train_label.shape, Y_train.shape)

(2048, 224, 224, 1) (22560, 224, 224, 1)
(22560, 26) (22560, 10)


In [ ]:
def train_test_model(X_train_label, X_train_pixel, Y_train):
    print(ModelName)
    with tf.device('/device:GPU:0'):
        resnet50 = tf.keras.applications.InceptionResNetV2(include_top=False, weights=None, input_tensor=None, input_shape=(N_dim,N_dim,1),pooling=None)
        cnn_mid = layers.GlobalAveragePooling2D()(resnet50.output)
        cnn_out = layers.Dense(128, activation = 'relu')(cnn_mid)

        dense_input = layers.Input(shape=(26,))
        dense_mid = layers.Dense(52, activation = 'relu')(dense_input)
        dense_output = layers.Dense(52, activation = 'relu')(dense_mid)
        dense_model = tf.keras.Model(inputs=dense_input, outputs=dense_output)

        concatenated = layers.concatenate([cnn_out, dense_model.output])
        concatenated = layers.Dense(32, activation='relu')(concatenated)
        concat_output = layers.Dense(10, activation='softmax')(concatenated)
        concat_model = tf.keras.models.Model([resnet50.input, dense_input], concat_output)
        concat_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
        history = concat_model.fit([X_train_pixel, X_train_label], Y_train, epochs=epochs_num,
                            verbose=verbose, shuffle=True)
    concat_model.save(path + ModelName + '.h5')
    print(
        f"CNN: Epochs={epochs_num:d}, " +
        f"Train accuracy={max(history.history['accuracy']):.5f}, "
    )
    return concat_model

model = train_test_model(X_train_label, X_train_pixel, Y_train)

InceptionResnetV2_bicubic224_VAE
Epoch 1/30
705/705 - 145s - loss: 1.0327 - accuracy: 0.6413
Epoch 2/30
705/705 - 145s - loss: 0.4669 - accuracy: 0.8365
Epoch 3/30
705/705 - 144s - loss: 0.3697 - accuracy: 0.8693
Epoch 4/30
705/705 - 145s - loss: 0.3150 - accuracy: 0.8858
Epoch 5/30
705/705 - 145s - loss: 0.2881 - accuracy: 0.8961
Epoch 6/30
705/705 - 145s - loss: 0.2719 - accuracy: 0.9011
Epoch 7/30
705/705 - 145s - loss: 0.2607 - accuracy: 0.9065
Epoch 8/30
705/705 - 145s - loss: 0.2547 - accuracy: 0.9072
Epoch 9/30
705/705 - 145s - loss: 0.2390 - accuracy: 0.9127
Epoch 10/30
705/705 - 145s - loss: 0.2408 - accuracy: 0.9137
Epoch 11/30
705/705 - 144s - loss: 0.2273 - accuracy: 0.9169
Epoch 12/30
705/705 - 144s - loss: 0.2428 - accuracy: 0.9132
Epoch 13/30
705/705 - 144s - loss: 0.2186 - accuracy: 0.9194
Epoch 14/30
705/705 - 144s - loss: 0.2280 - accuracy: 0.9170
Epoch 15/30
705/705 - 144s - loss: 0.2183 - accuracy: 0.9211
Epoch 16/30
705/705 - 144s - loss: 0.2197 - accuracy: 0.9215


In [6]:
def normalization(pixel_data):
    pixel_data = pixel_data.reshape(224 * 224)
    maxi = max(pixel_data)
    mini = min(pixel_data)
    if maxi == mini:
        pixel_data = np.array(list(map(lambda x: 0, pixel_data)))
    else:
        pixel_data = np.array(list(map(lambda x: (x - mini) / (maxi - mini), pixel_data)))
    return pixel_data.reshape(224,224,1)

class ComputerVision_Data_test():
    def __init__(self, dataframe, batch_size):
        self.pixel = dataframe.iloc[:, 2:].values.astype('int32').reshape(-1, 28, 28, 1)
        letter_hash = dict(zip(string.ascii_uppercase, [[1 if i == j else 0 for j in range(26)] for i in range(26)]))
        self.label =  np.array([letter_hash[letter] for letter in dataframe.iloc[:, 1]])
        self.batch_size = batch_size
        if len(dataframe) % batch_size == 0:
            self.length = len(dataframe) // batch_size
        else:
            self.length = len(dataframe) // batch_size + 1

    def __getitem__(self, idx):
        i = idx * self.batch_size
        if i + self.batch_size > len(self.pixel):
            index_window = [j for j in range(i, len(self.label))]
            target = self.pixel[i:]
        else:
            index_window = [j for j in range(i, i+self.batch_size)]
            target = self.pixel[i:i+self.batch_size]
        with tf.device('/device:GPU:0'):
            transformed = tf.image.resize(target, [224,224], method=tf.image.ResizeMethod.BICUBIC).numpy()
        pixel = np.asarray([normalization(elt) for elt in transformed])
        label = np.asarray([self.label[k] for k in index_window])

        return [pixel, label]

    def __len__(self):
        return self.length


test_data = pd.read_csv('/content/gdrive/My Drive/Dacon/ComputerVision/test.csv')
test_dataset = ComputerVision_Data_test(test_data, 1000)

In [ ]:
def get_progressbar_str(progress):
    MAX_LEN = 30
    BAR_LEN = int(MAX_LEN * progress)
    return ('Progress:[' + '=' * BAR_LEN +
            ('>' if BAR_LEN < MAX_LEN else '') +
            ' ' * (MAX_LEN - BAR_LEN) +
            '] %.1f%%' % (progress * 100.))

def split_testing(model,ModelName, test_dataset, batch_idxs):
    batch_size = test_dataset.batch_size
    submission = pd.read_csv(path_submission)
    inc = 0
    L = len(batch_idxs)
    for batch_idx in batch_idxs:
        target = test_dataset[batch_idx]
        real_idxs = [i for i in range(batch_size * batch_idx, batch_size * batch_idx + len(target[0])) ]
        with tf.device('/device:GPU:0'):
            predicted = model.predict(target)
        res = [int(np.argmax(x)) for x in predicted]
        for idx_temp, real in enumerate(real_idxs):
            submission.iloc[real,1] = res[idx_temp]
        inc += 1
        sys.stderr.write('\r\033[K' + get_progressbar_str(inc/L))
        sys.stderr.flush()
    sys.stderr.write('\n')
    sys.stderr.flush()
    
    submission.to_csv(path+'submission_'+ModelName+'.csv',index = False)

    return submission

#3
batch_idxs = [i for i in range(0, len(test_dataset))]
submission = split_testing(model,ModelName, test_dataset, batch_idxs)

Progress:[==============================] 100.0%


In [ ]:
def get_progressbar_str(progress):
    MAX_LEN = 30
    BAR_LEN = int(MAX_LEN * progress)
    return ('Progress:[' + '=' * BAR_LEN +
            ('>' if BAR_LEN < MAX_LEN else '') +
            ' ' * (MAX_LEN - BAR_LEN) +
            '] %.1f%%' % (progress * 100.))

def split_testing(model,ModelName, test_dataset, batch_idxs):
    batch_size = test_dataset.batch_size
    submission = pd.read_csv(path_submission)
    inc = 0
    L = len(batch_idxs)
    for batch_idx in batch_idxs:
        target = test_dataset[batch_idx]
        real_idxs = [i for i in range(batch_size * batch_idx, batch_size * batch_idx + len(target[0])) ]
        with tf.device('/device:GPU:0'):
            predicted = model.predict(target)
        res = [int(np.argmax(x)) for x in predicted]
        for idx_temp, real in enumerate(real_idxs):
            submission.iloc[real,1] = res[idx_temp]
        inc += 1
        sys.stderr.write('\r\033[K' + get_progressbar_str(inc/L))
        sys.stderr.flush()
    sys.stderr.write('\n')
    sys.stderr.flush()
    
    submission.to_csv(path+'submission_'+ModelName+'.csv',index = False)

    return submission

#3
batch_idxs = [i for i in range(0, len(test_dataset))]
submission = split_testing(model,ModelName, test_dataset, batch_idxs)

In [7]:
def get_progressbar_str(progress):
    MAX_LEN = 30
    BAR_LEN = int(MAX_LEN * progress)
    return ('Progress:[' + '=' * BAR_LEN +
            ('>' if BAR_LEN < MAX_LEN else '') +
            ' ' * (MAX_LEN - BAR_LEN) +
            '] %.1f%%' % (progress * 100.))

def split_testing(ModelNames, test_dataset, batch_idxs):
    batch_size = test_dataset.batch_size
    submission = pd.read_csv(path_submission)
    submission_temp = pd.DataFrame(index = np.arange(0,10),columns = np.arange(0,len(submission))).fillna(0)
    inc = 0
    L = len(batch_idxs) * len(ModelNames)
    for ModelName in ModelNames:
        model = tf.keras.models.load_model(path+ModelName+'.h5')
        for batch_idx in batch_idxs:
            target = test_dataset[batch_idx]
            real_idxs = [i for i in range(batch_size * batch_idx, batch_size * batch_idx + len(target[0])) ]
            with tf.device('/device:GPU:0'):
                predicted = model.predict(target)
            for idx_temp, real in enumerate(real_idxs):
                submission_temp.iloc[real,:] += predicted[idx_temp]
            inc += 1
            sys.stderr.write('\r\033[K' + get_progressbar_str(inc/L))
            sys.stderr.flush()
    sys.stderr.write('\n')
    sys.stderr.flush()
    submission.iloc[:,1] = submission_temp.apply(np.argmax,axis=1).values
        
    submission.to_csv(path+'submission_ensemble_'+str(len(ModelNames))+'.csv',index = False)

    return submission

batch_idxs = [i for i in range(0, len(test_dataset))]
submission = split_testing(['EfficientNetB5_bicubic224','InceptionResnetV2_bicubic224'], test_dataset, batch_idxs)

ValueError: ignored

In [10]:
def get_progressbar_str(progress):
    MAX_LEN = 30
    BAR_LEN = int(MAX_LEN * progress)
    return ('Progress:[' + '=' * BAR_LEN +
            ('>' if BAR_LEN < MAX_LEN else '') +
            ' ' * (MAX_LEN - BAR_LEN) +
            '] %.1f%%' % (progress * 100.))
batch_idxs = [i for i in range(0, len(test_dataset))]
ModelNames = ['EfficientNetB5_bicubic224','InceptionResnetV2_bicubic224']
batch_size = test_dataset.batch_size
submission = pd.read_csv(path_submission)
submission_temp = pd.DataFrame(columns = list(range(10)),index = list(range(len(submission)))).fillna(0)
inc = 0
L = len(batch_idxs) * len(ModelNames)
for ModelName in ModelNames:
    model = tf.keras.models.load_model(path+ModelName+'.h5')
    for batch_idx in batch_idxs:
        target = test_dataset[batch_idx]
        real_idxs = [i for i in range(batch_size * batch_idx, batch_size * batch_idx + len(target[0])) ]
        with tf.device('/device:GPU:0'):
            predicted = model.predict(target)
        for idx_temp, real in enumerate(real_idxs):
            submission_temp.iloc[real,:] += predicted[idx_temp]
        inc += 1
        sys.stderr.write('\r\033[K' + get_progressbar_str(inc/L))
        sys.stderr.flush()
sys.stderr.write('\n')
sys.stderr.flush()
submission.iloc[:,1] = submission_temp.apply(np.argmax,axis=1).apply(int).values
    
submission.to_csv(path+'submission_ensemble_'+str(len(ModelNames))+'.csv',index = False)

Progress:[==============================] 100.0%


AttributeError: ignored

In [11]:
submission.iloc[:,1] = submission_temp.apply(np.argmax,axis=1).apply(int).values

In [12]:
submission.to_csv(path+'submission_ensemble_'+str(len(ModelNames))+'.csv',index = False)